In [52]:
import sqlite3
import pandas as pd
import numpy as np
import json
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import knn_graph
from torch_geometric.nn import MessagePassing

In [19]:
with sqlite3.connect("data/storage.db") as conn:
    df_climbs=pd.read_sql_query('SELECT id, angle, grade, holds, ascents FROM climbs WHERE ascents > 1',conn, index_col='id')
df.drop_duplicates(subset=['angle','holds'],keep='first',inplace=True)
df['holds']=df['holds'].apply(lambda x: json.loads(x))
df['num_holds']=df['holds'].apply(lambda x: len(x))
df.sort_values(by='num_holds', ascending=False).head()

,angle,grade,holds,ascents,num_holds
id,,,,,
climb-157b42ce74fb,15,13.6667,"[[500, 0], [757, 1], [592, 2], [553, 2], [504,...",6,93
climb-a79d62cf4f21,25,18.4375,"[[500, 0], [757, 1], [592, 2], [553, 2], [504,...",16,93
climb-7cf74d016688,40,20.0000,"[[500, 0], [757, 1], [592, 2], [553, 2], [504,...",5,93
climb-3199b9e36cfa,45,22.0000,"[[500, 0], [757, 1], [592, 2], [553, 2], [504,...",2,93
climb-0f691a1f7004,30,20.6364,"[[500, 0], [757, 1], [592, 2], [553, 2], [504,...",11,93


In [23]:
with sqlite3.connect("data/storage.db") as conn:
    df_holds=pd.read_sql_query("SELECT * from holds WHERE wall_id = 'wall-443c15cd12e0'", conn, index_col='hold_index')
df_holds.head()

,id,wall_id,x,y,pull_x,pull_y,useability,is_foot,tags
hold_index,,,,,,,,,
390,hold-63eb3a62e9f5447,wall-443c15cd12e0,0.666667,0.333333,0.0,-1.0,0.5,1,None
392,hold-8750a752df80426,wall-443c15cd12e0,0.666667,1.666667,0.0,-1.0,0.5,1,None
394,hold-72a7570ab10c493,wall-443c15cd12e0,0.666667,3.000000,0.0,-1.0,0.5,0,None
396,hold-12b888837ef1422,wall-443c15cd12e0,0.666667,4.333333,0.0,-1.0,0.5,0,None
399,hold-59896feebf584bc,wall-443c15cd12e0,0.666667,6.333333,0.0,-1.0,0.5,0,None


In [53]:
#CONSTANTS
WALL_DIMS = [12,12]

#CLASSES
class ClimbingDiffusionModel(nn.Module):
    def __init__(self, num_scalar_features = 3, num_roles = 5, embedding_dim = 128, num_layers = 6):
        super.__init__()

        #Time Embedding
        self.time_mlp = nn.Sequential(
            nn.Linear(1, embedding_dim),
            nn.SiLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

        #Role Embedding
        self.role_embedding = nn.Embedding(num_roles, embedding_dim)

        #Scalar Feature Embedding
        self.scalar_feature_embedding = nn.Linear(num_scalar_features, embedding_dim)

        # Root Layer (Combine Time, Role, Scalar Embeddings)
        self.roots = nn.Sequential(
            nn.Linear(embedding_dim*3,embedding_dim),
            nn.SiLU(),
            nn.Linear(embedding_dim,embedding_dim)
        )

        # -- Trunk -- EGNN convolutional layer * n_layers
        self.trunk = nn.ModuleList([
            GravEGNNConv(
                node_dim = embedding_dim,
                vector_dim = 3,
                edge_dim = 0,
                hidden_dim = embedding_dim
            ) for _ in range(num_layers)
        ])

        # Output Heads (Scalar Feature Prediction, Role Prediction)
        self.feature_head = nn.Linear(embedding_dim, num_scalar_features)
        self.role_head = nn.Linear(embedding_dim, num_roles)

    def forward(self, x, v, scalars, roles, t, batch):
        # Embed time, features and combine them in roots.
        t_emb = self.time_mlp(t.unsqueeze(-1))
        t_nodes = t_emb[batch]
        
        role_emb = self.role_embedding(roles)
        scalar_emb = self.scalar_feature_embedding(scalars)

        
        # initial node-state h
        h = self.roots(torch.cat([scalar_emb, role_emb, t_nodes],dim=-1))
        
        # Construct the Graph (K-NN)
        edge_index = knn_graph(x,k=12,batch=batch)

        
        # Pass the data through the trunk
        for layer in self.trunk:
            h, x, v = layer(h, x, v, edge_index)
        
        # Decode scalar and role heads
        pred_r = self.role_head(h)
        pred_s = self.scalar_head(h)
        return {
            'pred_x': x,
            'pred_v': v,
            'pred_s': pred_s,
            'pred_r': pred_r,
        }

        
class ClimbingDiffusionTrainer(nn.Module):
    def __init__(self, model, num_roles=5,null_token=4):
        self.model = model
        self.num_roles = num_roles
        self.null_token = null_token
        self.loss_weights={'x': 1.0,'v': 2.0,'s': 0.5,'r': 1.0}

    def get_loss(self, batch):
        """Get the loss from a batch of real climbs padded with NULL tokens."""
        #Generate Random Sample from Batch
        batch_size = batch.num_graphs
        t = torch.rand(batch_size, device = batch.x.device)
        t_nodes = t[batch.batch]

        # 1. Forward Diffusion Process
        # Continuous Gaussian Diffusion (Noise)
        alpha_bar = torch.cos((t_nodes+0.008)/1.008*3.14159/2)**2
        noise_x = torch.randn_like(batch.pos)
        noise_v = torch.randn_like(batch.vec)
        noise_s = torch.randn_like(batch.scalars)

        x_noised = torch.sqrt(alpha_bar).unsqueeze(-1) * batch.pos + torch.sqrt(1-alpha_bar).unsqueeze(-1) * noise_x
        v_noised = torch.sqrt(alpha_bar).unsqueeze(-1) * batch.vec + torch.sqrt(1-alpha_bar).unsqueeze(-1) * noise_v
        s_noised = torch.sqert(alpha_bar).unsqueeze(-1) * batch.scalars + torch.sqrt(1-alpha_bar).unsqueeze(-1) * noise_s

        # 2. Discrete Absorbing State
        mask_prob = t_nodes
        # create a boolean mask determining which holds to set to the null_token (Random float in unif(1) < mask_prob)
        mask_decision = torch.rand_like(mask_prob) < mask_prob
        # noise the data by converting the changed roles to the null token.
        roles_noised = batch.roles.copy()
        roles_noised[mask_decision] = self.null_token
        # ensure all previously null roles are set to null. P(null|null)=1
        roles_noised[batch.roles == self.null_token] = self.null_token

        # 3. Predict the added noise using the model
        preds = self.model(x_noised, v_noised, s_noised, roles_noised, t, batch.batch)

        # 4. Calculate Loss
        #boolean mask over the null tokens to prevent them from affecting the loss function.
        is_real = (batch.roles != self.null_token).float().unsqueeze(-1)

        # Loss functions (Coordinate, Vector, Scalar loss). MSE over real points.
        loss_x = F.mse_loss(preds['pred_x']*is_real, batch.pos*is_real)
        loss_v = F.mse_loss(preds['pred_v']*is_real, batch.vec*is_real)
        loss_s = F.mse_loss(preds['pred_s']*is_real, batch.scalars*is_real)

        # Role Loss (Cross-Entropy)
        loss_r = F.cross_entropy(preds['pred_r'],batch.roles)
        
        # Total Loss (weighted sum of x,v,s,r)
        total_loss = loss_weights['x']*loss_x + loss_weights['v']*loss_v+loss_weights['s']*loss_s+loss_weights['r']*loss_r

        return total_loss

In [48]:
1.008/1.008*3.14159/2 

1.570795

In [49]:
3.14159/2

1.570795